In [1]:
# import common library
import numpy as np
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import time
import sys

# import custom module
import import_ipynb

from CompNet import CompNet
from CompNetParametric import CompNetParametric
from PairGen import PairGen

In [2]:
# load raw dataset
datasetStr = "FashionMNIST"
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

X_train = X_train.astype('float32') / 255;
X_test = X_test.astype('float32') / 255;

# create pair generator
pairgen = PairGen(X_train, y_train, X_test, y_test)
ds_train = pairgen.ds_train
ds_valid = pairgen.ds_valid
ds_test = pairgen.ds_test

In [3]:
X_test_comp_left_list = []
X_test_comp_right_list = []
y_test_comp_list = []

for batch in ds_test.repeat().take(8):
    for icnt in range(32):
        X_test_comp_left_list.append(batch[0][0][icnt])
        X_test_comp_right_list.append(batch[0][1][icnt])
        y_test_comp_list.append(batch[1][icnt][0])
        
X_test_comp_left = tf.convert_to_tensor(np.array(X_test_comp_left_list))
X_test_comp_right = tf.convert_to_tensor(np.array(X_test_comp_right_list))
y_test_comp = tf.convert_to_tensor(np.array(y_test_comp_list))

X_test = np.array([X_test_comp_left, X_test_comp_right])

print(np.shape(X_test))

(2, 256, 28, 28, 1)


In [ ]:
import os
rootDir = "./models"
dataDirList = os.listdir(rootDir)
for dataDirName in dataDirList:
    if dataDirName.lower() != datasetStr.lower(): continue
    dirList = os.listdir(rootDir + "/" + dataDirName)
    for dirName in dirList:
        tf.keras.backend.clear_session()

        modelname = rootDir + "/" + dataDirName + "/" + dirName + "/whole.tfmodel"
        print(modelname)
        model = tf.keras.models.load_model(modelname)

        y_pred_test = np.squeeze(model.predict(X_test))

        pred_pos = np.size(np.where(y_pred_test > 0.5))
        pred_neg = np.size(np.where(y_pred_test <= 0.5))
        pred_total = np.size(y_pred_test)

        truth_pos = np.size(np.where(y_test_comp > 0.5))
        truth_neg = np.size(np.where(y_test_comp <= 0.5))
        truth_total = np.size(y_test_comp)

        predStr = str(pred_pos) + "_" + str(pred_neg) + "_" + str(pred_total)
        truthStr = str(truth_pos) + "_" + str(truth_neg) + "_" + str(truth_total)

        predminmax = str(np.min(y_pred_test)) + "_" + str(np.mean(y_pred_test)) + "_" + str(np.max(y_pred_test))
        truthminmax = str(np.min(y_test_comp)) + "_" + str(np.mean(y_test_comp)) + "_" + str(np.max(y_test_comp))

        file = open(rootDir + "/" + dataDirName + "/" + dirName + "/pred_pos_neg_total_" + predStr + ".tag", "w")
        file.close()

        file = open(rootDir + "/" + dataDirName + "/" + dirName + "/truth_pos_neg_total_" + truthStr + ".tag", "w")
        file.close()

        file = open(rootDir + "/" + dataDirName + "/" + dirName + "/pred_min_mean_max_" + predminmax + ".tag", "w")
        file.close()

        file = open(rootDir + "/" + dataDirName + "/" + dirName + "/truth_min_mean_max_" + truthminmax + ".tag", "w")
        file.close()